In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os 
import glob
from tqdm import tqdm 
import math

In [4]:
def binarize_traces(df, threshold):
    """
    Return a copy of df where every value < threshold becomes 0,
    and every value >= threshold becomes 1.
    """
    return (df >= threshold).astype(int)

def get_event_rate(df, samplingRate):
    """
    Given a binarized DataFrame `df` of shape (n_frames, n_cells),
    compute, for each cell (column), the number of “events” in each
    forward-looking 1-second window. Returns a DataFrame of shape
    (n_frames - samplingRate + 1, n_cells).
    """
    # 1) grab the raw values (frames × cells)
    arr = df.values.astype(int)  
    # 2) build a “boxcar” kernel of length = samples per second
    kernel = np.ones(samplingRate, dtype=int)
    # 3) convolve along the time-axis for each cell (axis=0)
    #    mode='valid' gives you only positions where the full window fits
    rates = np.apply_along_axis(
        lambda col: np.convolve(col, kernel, mode='valid'),
        axis=0,
        arr=arr
    )
    # 4) build a new index so that row i corresponds to window df.index[i : i+samplingRate]
    #    if your original df.index is RangeIndex starting at 0, this is simply 0..n-sR
    new_index = df.index[: rates.shape[0]]
    return pd.DataFrame(rates, index=new_index, columns=df.columns)

def get_indices_by_speed_bins(df, speed_col, bins):
    """
    For each (low, high) in `bins`, find the row-indices where
    low <= df[speed_col] < high (for high == inf, just low <=).
    
    Returns
    -------
    dict
        keys   : tuple (low, high)
        values : list of row-index labels in df
    """
    bin_indices = {}
    for low, high in bins:
        if np.isinf(high):
            mask = df[speed_col] >= low
        else:
            mask = (df[speed_col] >= low) & (df[speed_col] < high)
        bin_indices[(low, high)] = df.index[mask].tolist()
    return bin_indices

def avg_event_rate_by_speed_bins(df, idx_by_bin, speed_col='linearSpeedcmPerSecond'):
    """
    Given:
      • df            : DataFrame with one column per cell (instantaneous rates)
                        plus a `speed_col`.
      • idx_by_bin    : dict mapping (low, high) tuples → list of row-indices
                        where speed ∈ [low, high) (or >= low if high is inf).
      • speed_col     : name of the speed column in df (will be dropped).
    Returns:
      • avg_df        : DataFrame indexed by your (low, high) bins,
                        columns are the cell-names, entries are the mean
                        event-rate of that cell over all frames in that bin.
    """
    # 1) Identify all “cell” columns (everything except the speed column)
    event_cols = [c for c in df.columns if c != speed_col]

    # 2) Make a DataFrame to hold means; use a MultiIndex of your bin tuples
    bin_index = pd.MultiIndex.from_tuples(idx_by_bin.keys(), names=['low','high'])
    avg_df = pd.DataFrame(index=bin_index, columns=event_cols, dtype=float)

    # 3) For each bin, pull those rows and take the column‐wise mean
    for bin_range, idxs in idx_by_bin.items():
        if len(idxs)>0:
            avg_df.loc[bin_range] = df.loc[idxs, event_cols].mean()
        else:
            avg_df.loc[bin_range] = np.nan

    return avg_df


def apply_spatial_filter(
    df,
    frame_rate: int,
    square_box_threshold: float,
    coord_cols=('X_coor','Y_coor'),
    vel_col='velocity_PixelsPerSec'
):
    """
    Slide a window of `frame_rate` frames over the trajectory;
    whenever the animal stays within a ±square_box_threshold pixel
    box (around the start‐of‐window position) for the whole window,
    zero out the velocity over that window.

    Returns a new numpy array of filtered velocities.
    """
    X = df[coord_cols[0]].to_numpy()
    Y = df[coord_cols[1]].to_numpy()
    vel = df[vel_col].to_numpy().copy()
    n = len(df)
    w = frame_rate

    for i in range(n):
        end = min(i + w, n)
        cx, cy = X[i], Y[i]
        Xw, Yw = X[i:end], Y[i:end]

        # check if all points stay in the box
        if ((Xw >= cx - square_box_threshold) & (Xw <= cx + square_box_threshold)).all() \
        and ((Yw >= cy - square_box_threshold) & (Yw <= cy + square_box_threshold)).all():
            vel[i:end] = 0

    return vel

In [5]:
#
baseDirPath = '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/'
sessions = ['1_24_mc_989_1_4_25_Run12', 
            '1_24_motion_corrected_989_1_1_25_Run12', 
            '1_24_motion_corrected_989_1_2_25_Run12',
           ] 

In [6]:
baseDirPath

'/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/'

In [7]:
glob.glob(os.path.join(baseDirPath, '*cellTracesAlignedToTracking.csv'))

['/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_motion_corrected_989_1_2_25_Run12cellTracesAlignedToTracking.csv',
 '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_mc_989_1_4_25_Run12cellTracesAlignedToTracking.csv',
 '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_motion_corrected_989_1_1_25_Run12cellTracesAlignedToTracking.csv']

In [8]:
dataByMouse = {}

#path to eZTrack data
for session in sessions: 
    print(session)
    dirPath = baseDirPath
    alignedFile = glob.glob(os.path.join(dirPath, '*cellTracesAlignedToTracking.csv'))[0]
    print(alignedFile)
    
    #load calcium signal aligned to tracking data 
    alignedTraces = pd.read_csv(alignedFile)
    #separate out location data 
    alignedTracesLocationData = alignedTraces[['closestBehavCamFrameIdx', 'X_coor', 'Y_coor']]
    #separate out cell traces
    aligned_cell_traces = alignedTraces.loc[:, alignedTraces.columns.str.startswith('cell_')]
    #threshold signal 
    firingThresholdSD = 2.5
    signalPeaks = binarize_traces(aligned_cell_traces, firingThresholdSD)
    samplingRate = 20 
    instantaneousEventRate = get_event_rate(signalPeaks, samplingRate)

    #this calculates the velocity in pixels per second from both the x and y coordinates 
    alignedTraces['velocity_PixelsPerSec'] = (np.hypot(
        alignedTraces['X_coor'].diff(),
        alignedTraces['Y_coor'].diff()
        ) * samplingRate
    ).fillna(0)

    #take just the X coordinate column and calculate a X velocity (in pixels)
    samplingRate = 20 
    linearSpeed = abs(alignedTraces['X_coor'].diff() * samplingRate)
    linearSpeed.fillna(0, inplace=True)
    pixelsPercm = 3.8
    linearSpeedcmPerSecond = linearSpeed / pixelsPercm

    #append to event rate dataframe 
    instantaneousEventRate['linearSpeedcmPerSecond'] = linearSpeedcmPerSecond

    #perform "spatial filter" on 'velocity_PixelsPerSec' trace 
    frameRate = 5
    squareBoxThreshold = 4

    alignedTraces['velocitySpatialFiltered'] = apply_spatial_filter(
        alignedTraces,
        frame_rate=frameRate,
        square_box_threshold=squareBoxThreshold,
        coord_cols=('X_coor','Y_coor'),
        vel_col='velocity_PixelsPerSec'   # change this if your velocity column is named differently
    )

    #perform temporal filter on speed trace
    window = int(samplingRate/5)  
    alignedTraces['velocitySmooth'] = (alignedTraces['velocitySpatialFiltered'].rolling(window, center=True, min_periods=1).mean())

    #convert to cm 
    velocity2dPer_cm_Filtered = alignedTraces['velocitySpatialFiltered'] / pixelsPercm

    #append to event rate dataframe 
    alignedTraces['velocity2dSpatialFiltered'] = velocity2dPer_cm_Filtered
    instantaneousEventRate['velocity2dSpatialFiltered'] = velocity2dPer_cm_Filtered

    #calculate, for each cell, event rate at different speed bins 
    bins = [(0, 0.25), (0.25, 1), (1, 2.5), (2.5, 5), (5, 10), (10, np.inf)]

    idx_by_bin = get_indices_by_speed_bins(instantaneousEventRate, 'velocity2dSpatialFiltered', bins)
    #avg rates by speed 
    avg_rates = avg_event_rate_by_speed_bins(instantaneousEventRate, idx_by_bin, speed_col='velocity2dSpatialFiltered')
    avg_rates = avg_rates.drop(columns=['linearSpeedcmPerSecond'])

    alignedTraces.to_csv(dirPath+'alignedTracesWithVelocity.csv')
    instantaneousEventRate.to_csv(dirPath+'alignedEventRate.csv') 
    
    dataByMouse[session]={'alignedTraces':alignedTraces,
                          'instantaneousEventRate':instantaneousEventRate,
                          'indiciesInBins':idx_by_bin, 
                          'firingRatesBySpeedBin':avg_rates}
    
    print('done')

1_24_mc_989_1_4_25_Run12
/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_motion_corrected_989_1_2_25_Run12cellTracesAlignedToTracking.csv
done
1_24_motion_corrected_989_1_1_25_Run12
/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_motion_corrected_989_1_2_25_Run12cellTracesAlignedToTracking.csv
done
1_24_motion_corrected_989_1_2_25_Run12
/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/firingRateAnalysis/01_2025_m989/1_24_motion_corrected_989_1_2_25_Run12cellTracesAlignedToTracking.csv
done


In [9]:
dfs = [v['firingRatesBySpeedBin'] for v in dataByMouse.values()]
all_rates = pd.concat(dfs, axis=1)
all_rates.to_csv(baseDirPath+'binnedFiringRate.csv')
mean_firing_rate = all_rates.mean(axis=1)
std_firing_rate = all_rates.std(axis=1)/math.sqrt(all_rates.shape[1])

In [10]:
mean_firing_rate

low    high 
0.00   0.25     0.482235
0.25   1.00     0.538961
1.00   2.50     0.559740
2.50   5.00     0.651786
5.00   10.00    0.608635
10.00  inf      0.564486
dtype: float64

In [11]:
std_firing_rate

low    high 
0.00   0.25     0.010834
0.25   1.00     0.137502
1.00   2.50     0.053892
2.50   5.00     0.055031
5.00   10.00    0.037471
10.00  inf      0.028524
dtype: float64

In [12]:
# 1) compute the per-frame average of every numeric column
binned = (
    alignedTraces
      .groupby('closestBehavCamFrameIdx')           # group by frame-idx
      .mean(numeric_only=True)                     # mean of only number-dtype cols
)

In [13]:
binned

,cell_0,cell_2,cell_3,cell_4,cell_5,cell_7,cell_9,cell_10,cell_11,cell_12,...,Frame Number,Time Stamp (ms),Buffer Index,X_coor,Y_coor,Distance_px,velocity_PixelsPerSec,velocitySpatialFiltered,velocitySmooth,velocity2dSpatialFiltered
closestBehavCamFrameIdx,,,,,,,,,,,,,,,,,,,,,
0,-0.204950,-1.209400,0.955802,-0.431357,-0.248612,3.025063,-0.533682,0.620557,-0.263121,1.154552,...,0.0,-21.0,0.0,17.653983,26.596213,0.000000,0.000000,0.0,0.0,0.0
1,-0.222556,-1.081069,0.949877,-0.949637,-0.513922,3.267748,-0.509944,0.164223,0.052334,0.667569,...,1.5,57.5,0.0,16.995678,26.576225,0.658608,6.586079,0.0,0.0,0.0
2,-0.522714,-0.943719,0.697907,-0.805307,-0.184304,3.224350,-1.268067,0.378639,-0.061966,0.577634,...,3.0,132.0,0.0,16.979260,26.535482,0.043927,0.878537,0.0,0.0,0.0
3,-0.352180,-1.018692,0.789387,-0.588454,-0.216616,3.428789,-0.769319,0.387084,0.203337,0.846181,...,4.5,207.0,0.0,17.057075,26.554052,0.080000,0.800005,0.0,0.0,0.0
4,-0.816209,-1.115295,0.729587,-1.026050,-0.395660,3.114093,-0.755844,0.456542,0.269480,0.583235,...,6.0,282.0,0.0,17.191726,26.696713,0.196172,3.923432,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17208,-2.324064,1.452903,-1.732920,1.216892,-2.048625,-1.192324,0.089814,-1.145851,-2.300799,-1.543674,...,23689.0,1199759.0,0.0,13.608081,26.977223,0.133628,2.672564,0.0,0.0,0.0
17209,-1.986863,1.362319,-1.704784,0.942141,-2.146622,-1.374600,-0.235652,-1.184595,-1.787047,-1.596993,...,23690.0,1199810.0,0.0,13.702463,26.986881,0.094874,1.897490,0.0,0.0,0.0
17210,-1.379248,1.605775,-1.706273,0.790389,-1.999436,-1.212737,0.231648,-1.195407,-1.792171,-1.745209,...,23691.5,1199886.0,0.0,13.889956,26.875796,0.217930,2.179302,0.0,0.0,0.0


In [ ]:
##validate with video overlay  
#small video 

# ── CONFIG ─────────────────────────────────────────────────────────────────────

dirPath      = '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/2025.4/m328/2025_04_10_328_15_59_58_b2/My_WebCam/'
rotatedVideo = 'video_output.avi'
input_vid    = os.path.join(dirPath, rotatedVideo)
output_vid   = os.path.join(dirPath, 'with_velocity_overlay.mp4')

# your velocity array (length == # frames)
vel_array = binned['velocity2dSpatialFiltered'].to_numpy()

# your per-behavior-frame X positions (make sure it's indexed by frame 0…n-1)
# e.g. binned = binned.reset_index().set_index('frame')
x_pos = binned['X_coor'].to_numpy()

# height of the black bar you want to add
bar_h = 50

# ── SET UP I/O ─────────────────────────────────────────────────────────────────

cap      = cv2.VideoCapture(input_vid)
fps      = cap.get(cv2.CAP_PROP_FPS)
w        = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h        = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# choose a codec that FIJI likes for AVIs
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out    = cv2.VideoWriter(output_vid, fourcc, fps, (w, h + bar_h))

# text & dot styling
font        = cv2.FONT_HERSHEY_SIMPLEX
font_scale  = 1
thickness   = 2
margin      = 10
dot_radius  = 5

# ── PROCESS FRAMES ──────────────────────────────────────────────────────────────

for frame_idx in tqdm(range(n_frames), desc="Adding overlay"):
    ret, frame = cap.read()
    if not ret:
        break

    # 1) create the padded frame
    bar       = np.zeros((bar_h, w, 3), dtype=np.uint8)
    new_frame = np.vstack((frame, bar))

    # 2) overlay the velocity text in the bar (left‐aligned)
    vel_text = f"{vel_array[frame_idx-1]:.2f} cm/s"
    (tw, th), _ = cv2.getTextSize(vel_text, font, font_scale, thickness)
    text_x = margin
    text_y = h + margin + th
    cv2.putText(
        new_frame,
        vel_text,
        (text_x, text_y),
        font,
        font_scale,
        (255, 255, 255),
        thickness,
        cv2.LINE_AA
    )

    # 3) draw the dot at the mouse’s X in the center of the bar
    xm = int(np.clip(x_pos[frame_idx-1], 0, w-1))
    y_dot = h + bar_h // 2
    cv2.circle(new_frame, (xm, y_dot), dot_radius, (255, 255, 255), -1)

    out.write(new_frame)

# ── CLEAN UP ────────────────────────────────────────────────────────────────────

cap.release()
out.release()
print("Done – saved with overlay to:", output_vid)